In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

In [55]:
def avg_encoder(colum):
    dic = {}
    for i in data[colum].drop_duplicates():
        summ = data.loc[data[colum] == i, 'price'].sum()
        num = data.loc[data[colum] == i, 'price'].count()
        dic[i] = str(summ//num)

    for key,value in dic.items():
        #print(key, ' ' , value)
        data[colum] = data[colum].str.replace(key,value,regex=True)
    data[colum] = pd.to_numeric(data[colum], downcast="integer")

In [59]:
data = pd.read_csv('airline-price-prediction.csv').dropna(how='any',inplace=False) #Load players data & Drop missing values
data

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,5/3/2022,Vistara,UK,812,9:45,10h 10m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...","56,588"
1,18-03-2022,Vistara,UK,975,5:45,06h 30m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}","59,649"
2,9/3/2022,GO FIRST,G8,7537,14:30,08h 10m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}","3,916"
3,15-03-2022,GO FIRST,G8,287,10:40,09h 40m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}","6,031"
4,22-03-2022,Vistara,UK,826,12:30,07h 25m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}","7,081"
...,...,...,...,...,...,...,...,...,...,...,...
240203,25-03-2022,Air India,AI,430,9:55,09h 25m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,19:20,economy,"{'source': 'Chennai', 'destination': 'Hyderabad'}","3,979"
240204,15-02-2022,Air India,AI,489,19:10,17h 20m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,12:30,business,"{'source': 'Delhi', 'destination': 'Kolkata'}","42,771"
240205,23-02-2022,Indigo,6E,2036,22:50,02h 55m,non-stop,1:45,economy,"{'source': 'Delhi', 'destination': 'Bangalore'}","7,425"
240206,11/2/2022,Vistara,UK,813,17:35,28h 15m,1-stop\r\n\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,21:50,economy,"{'source': 'Delhi', 'destination': 'Mumbai'}","14,093"


In [60]:
#price
data['price'] = pd.to_numeric(data['price'].str.replace(',',''), downcast="integer")

In [61]:
#Date
data[['day','month','year']] = data['date'].str.replace('-','/').str.split('/', expand=True)
data = data.drop(['date'], axis=1)
data = data.drop(['year'], axis=1) # all at 2022

In [62]:
#time_taken
h = pd.to_numeric(data['time_taken'].str.split(' ', expand=True)[0].str[:-1] , downcast="integer")
m = pd.to_numeric(data['time_taken'].str.split(' ', expand=True)[1].str[:-1] , downcast="integer")
data['time_taken'] = m + h*60

In [63]:
#dep_time
data[['dep_h','dep_m']] = data['dep_time'].str.split(':', expand=True)
data = data.drop(['dep_time'], axis=1)

In [64]:
#arr_time
data[['arr_h','arr_m']] = data['arr_time'].str.split(':', expand=True)
data = data.drop(['arr_time'], axis=1)

In [65]:
#type
data['type'] = LabelEncoder().fit(list(data['type'].values)).transform(list(data['type'].values))

In [66]:
#stop
data['stop'] = data['stop'].str.split('p', expand=True)[0].str.replace('+','',regex=True)
data['stop'] = LabelEncoder().fit(list(data['stop'].values)).transform(list(data['stop'].values))
#avg_encoder('stop')

In [67]:
#route
avg_encoder("route")

In [68]:
#airline
avg_encoder("airline")
#avg_encoder("ch_code") # ch_code same value like airline
data = data.drop(['ch_code'], axis=1)

In [69]:
#convert all to integer
for coulm in data.columns:
        data[coulm] = pd.to_numeric(data[coulm],downcast="integer")

In [70]:
#new fetures
data['airline&type'] = (data['airline'] + data['type']).apply(str)
avg_encoder('airline&type')

In [72]:
data

,airline,num_code,time_taken,stop,type,route,day,month,dep_h,dep_m,arr_h,arr_m,airline&type,total_price
0,30404,812,610.0,0,0,21218,5,3,9,45,19,55,55481,56588
1,30404,975,390.0,0,0,17459,18,3,5,45,12,15,55481,59649
2,5645,7537,490.0,0,1,20968,9,3,14,30,22,40,5645,3916
3,5645,287,580.0,0,1,18626,15,3,10,40,20,20,5645,6031
4,30404,826,445.0,0,1,22757,22,3,12,30,19,55,7805,7081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240203,23441,430,565.0,0,1,21687,25,3,9,55,19,20,7307,3979
240204,23441,489,1040.0,0,0,20558,15,2,19,10,12,30,47162,42771
240205,5322,2036,175.0,2,1,17864,23,2,22,50,1,45,5322,7425
240206,30404,813,1695.0,0,1,19273,11,2,17,35,21,50,7805,14093


In [71]:
data['total_price'] = data['price']
data = data.drop(['price'], axis=1)
data.corr()

,airline,num_code,time_taken,stop,type,route,day,month,dep_h,dep_m,arr_h,arr_m,airline&type,total_price
airline,1.000000,-0.334994,0.363515,-0.165756,-0.444900,0.066837,0.000384,-0.026858,0.037515,0.065630,0.122503,0.120443,0.500067,0.472026
num_code,-0.334994,1.000000,-0.134980,0.074509,0.217631,-0.028637,-0.009804,0.001813,-0.015159,0.014880,-0.002875,-0.041387,-0.223387,-0.213285
time_taken,0.363515,-0.134980,1.000000,-0.474561,-0.139256,0.130227,-0.002953,-0.037083,0.142022,0.006772,-0.023062,0.051050,0.149936,0.205602
stop,-0.165756,0.074509,-0.474561,1.000000,0.099140,-0.077487,0.004247,-0.008580,0.012051,-0.008140,-0.048643,-0.028970,-0.105512,-0.202568
type,-0.444900,0.217631,-0.139256,0.099140,1.000000,-0.031283,-0.001844,0.014475,-0.039036,-0.014563,-0.051034,-0.041392,-0.993491,-0.937784
route,0.066837,-0.028637,0.130227,-0.077487,-0.031283,1.000000,-0.002444,-0.006809,0.006904,-0.027965,0.045652,-0.021494,0.035676,0.091832
day,0.000384,-0.009804,-0.002953,0.004247,-0.001844,-0.002444,1.000000,-0.262064,-0.001726,-0.005670,-0.004750,-0.002183,0.002457,-0.002118
month,-0.026858,0.001813,-0.037083,-0.008580,0.014475,-0.006809,-0.262064,1.000000,-0.001612,-0.005827,-0.019796,-0.005772,-0.014766,-0.092577
dep_h,0.037515,-0.015159,0.142022,0.012051,-0.039036,0.006904,-0.001726,-0.001612,1.000000,0.058892,-0.157754,0.049871,0.039830,0.031570
dep_m,0.065630,0.014880,0.006772,-0.008140,-0.014563,-0.027965,-0.005670,-0.005827,0.058892,1.000000,0.002257,0.047748,0.030160,0.029075


In [48]:
Y=data['total_price'] #Goal
X=np.array(data.iloc[:,[0,12]]) #Futures
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size= 0.20 ,shuffle=True) # train_test_split
poly = PolynomialFeatures(degree=2) #declare poly transformer
x_train = poly.fit_transform(x_train) #transforms features to higher degree
x_test = poly.fit_transform(x_test) #transforms features to higher degree
leaner = LinearRegression(normalize=True).fit(x_train, y_train) #declare leaner model & Normalize & fit
y_train_predicted = leaner.predict(x_train) # predict train data-set
y_test_predicted = leaner.predict(x_test) # predict test data-set

print('MSE of train = ', metrics.mean_squared_error(y_train, y_train_predicted))
print('MSE of test = ', metrics.mean_squared_error(y_test, y_test_predicted))
print('true_value = ',np.asarray(y_test)[12])
print('prediction_value = ',y_test_predicted[12])

MSE of train =  55948724.8506597
MSE of test =  56982479.4728406
true_value =  6461
prediction_value =  7811.823694552018


In [579]:
#data['source'] =data['route'].str.split(',', expand=True)[0].str.split(':', expand=True)[1].str.split("'", expand=True)[1]
#data['destination'] =data['route'].str.split(',', expand=True)[1].str.split(':', expand=True)[1].str.split("'", expand=True)[1]
#data = data.drop(['route'], axis=1)
#data.corr()

,num_code,source,destination,total_price
num_code,1.000000,-0.012139,-0.019477,-0.213285
source,-0.012139,1.000000,-0.250173,0.050147
destination,-0.019477,-0.250173,1.000000,0.056765
total_price,-0.213285,0.050147,0.056765,1.000000
